In [13]:
import json
import plotly.express as px
import datetime
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
import pickle
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.utils import shuffle
from keras.utils.np_utils import to_categorical

In [14]:
signals_df = pd.read_csv("csvs/signals.csv")
signals_df.reset_index(drop=True, inplace=True)

In [15]:
signals_df.head()

,url,has_login_pattern_on_form,has_register_pattern_on_form,has_newsletter_pattern_on_form,has_forgot_pattern_on_form,has_forgot_password_pattern_in_text_content,has_forgot_password_pattern_in_attrs,has_forgot_password_pattern_in_text_content_on_form,has_forgot_password_pattern_in_attrs_on_form,has_forgot_pattern_in_attrs,...,has_already_pattern,has_already_pattern_on_form,not_have_pattern,not_have_pattern_on_form,has_newsletter_pattern.2,has_reset_pattern_on_url,has_newsletter_pattern_on_url,page_type,Sum,type
0,https://100things2do.ca/organized/,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,n,0,False
1,https://10bet.co.tz/,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,n,2,False
2,https://126dbs.com/2021/04/,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,n,0,False
3,https://1cloudfile.com/register,True,True,False,False,False,False,False,False,False,...,True,False,True,False,False,False,False,r,14,True
4,https://1cloudfile.com/account/login,True,False,False,False,True,True,True,True,False,...,False,False,True,False,False,False,False,l,11,True


In [16]:
def get_multi_class_type(type):
    # n means not_login_signup, e means newsletter 
    if((type=='n')|(type=='e')):
        return 0
    # l means login, b means both login and signup
    if((type=='l')|(type=='b')):
        return 1
    # r means register(signup) page
    if((type=='r')):
        return 2

In [17]:
signals_df['type'] = signals_df['page_type'].map(get_multi_class_type)

In [18]:
print("Num of login pages: %s,\nNum of signup pages: %s,\nNum of non-login-signup pages: %s" % (len(signals_df[signals_df.type==1]), len(signals_df[signals_df.type==2]), len(signals_df[signals_df.type==0])))

Num of login pages: 1299,
Num of signup pages: 973,
Num of non-login-signup pages: 2453


In [19]:
# Ignore pages where we got less than 3 signals on login and signup pages
only_features_detected = signals_df[((signals_df.Sum>3) & (signals_df.type==1) ) |((signals_df.Sum>3) & (signals_df.type==2) ) | (signals_df.type==0)]
only_features_detected.head()

,url,has_login_pattern_on_form,has_register_pattern_on_form,has_newsletter_pattern_on_form,has_forgot_pattern_on_form,has_forgot_password_pattern_in_text_content,has_forgot_password_pattern_in_attrs,has_forgot_password_pattern_in_text_content_on_form,has_forgot_password_pattern_in_attrs_on_form,has_forgot_pattern_in_attrs,...,has_already_pattern,has_already_pattern_on_form,not_have_pattern,not_have_pattern_on_form,has_newsletter_pattern.2,has_reset_pattern_on_url,has_newsletter_pattern_on_url,page_type,Sum,type
0,https://100things2do.ca/organized/,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,n,0,0
1,https://10bet.co.tz/,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,n,2,0
2,https://126dbs.com/2021/04/,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,n,0,0
3,https://1cloudfile.com/register,True,True,False,False,False,False,False,False,False,...,True,False,True,False,False,False,False,r,14,2
4,https://1cloudfile.com/account/login,True,False,False,False,True,True,True,True,False,...,False,False,True,False,False,False,False,l,11,1


In [20]:
print("After filtering by minumum 3 signals received pages: \nNum of login pages: %s,\nNum of signup pages: %s,\nNum of non-login-signup pages: %s" % (len(only_features_detected[only_features_detected.type==1]), len(only_features_detected[only_features_detected.type==2]), len(only_features_detected[only_features_detected.type==0])))

After filtering by minumum 3 signals received pages: 
Num of login pages: 1299,
Num of signup pages: 973,
Num of non-login-signup pages: 2453


In [22]:
predictors = only_features_detected.iloc[:, 1:89]
target = only_features_detected['type']
encoder = LabelEncoder()
encoder.fit(target)
Y = encoder.transform(target)
Y = to_categorical(Y)
X_train,X_test,Y_train,Y_test = train_test_split(predictors,Y,test_size=0.33)

In [23]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=88))
model.add(Dense(16,activation='relu'))
model.add(Dense(8,activation='relu'))
model.add(Dense(3,activation='sigmoid'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
history = model.fit(X_train,Y_train,epochs=200)

Epoch 1/200
99/99 [==============================] - 0s 1ms/step - loss: 0.8933 - accuracy: 0.7476
Epoch 2/200
99/99 [==============================] - 0s 1ms/step - loss: 0.3788 - accuracy: 0.9419
Epoch 3/200
99/99 [==============================] - 0s 2ms/step - loss: 0.1218 - accuracy: 0.9694
Epoch 4/200
99/99 [==============================] - 0s 3ms/step - loss: 0.0857 - accuracy: 0.9760
Epoch 5/200
99/99 [==============================] - 0s 3ms/step - loss: 0.0746 - accuracy: 0.9760
Epoch 6/200
99/99 [==============================] - 0s 3ms/step - loss: 0.0670 - accuracy: 0.9804
Epoch 7/200
99/99 [==============================] - 0s 3ms/step - loss: 0.0628 - accuracy: 0.9810
Epoch 8/200
99/99 [==============================] - 0s 3ms/step - loss: 0.0599 - accuracy: 0.9810
Epoch 9/200
99/99 [==============================] - 0s 3ms/step - loss: 0.0573 - accuracy: 0.9836
Epoch 10/200
99/99 [==============================] - 0s 3ms/step - loss: 0.0552 - accuracy: 0.9839
Epoch 11/

99/99 [==============================] - 0s 3ms/step - loss: 0.0116 - accuracy: 0.9940
Epoch 84/200
99/99 [==============================] - 0s 2ms/step - loss: 0.0140 - accuracy: 0.9930
Epoch 85/200
99/99 [==============================] - 0s 3ms/step - loss: 0.0120 - accuracy: 0.9940
Epoch 86/200
99/99 [==============================] - 0s 3ms/step - loss: 0.0124 - accuracy: 0.9937
Epoch 87/200
99/99 [==============================] - 0s 3ms/step - loss: 0.0128 - accuracy: 0.9937
Epoch 88/200
99/99 [==============================] - 0s 3ms/step - loss: 0.0131 - accuracy: 0.9943
Epoch 89/200
99/99 [==============================] - 0s 3ms/step - loss: 0.0120 - accuracy: 0.9927
Epoch 90/200
99/99 [==============================] - 0s 2ms/step - loss: 0.0115 - accuracy: 0.9946
Epoch 91/200
99/99 [==============================] - 0s 3ms/step - loss: 0.0115 - accuracy: 0.9937
Epoch 92/200
99/99 [==============================] - 0s 3ms/step - loss: 0.0115 - accuracy: 0.9937
Epoch 93/200


In [ ]:
scores = model.evaluate(X_test, Y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
X_test.shape

In [ ]:
Y_pred_nn = model.predict(X_test)

In [ ]:
import numpy as np
y_actual = np.argmax(Y_test, axis=1)

In [ ]:
import numpy as np
y = np.argmax(Y_pred_nn, axis=1)

In [ ]:
print("classification Report:\n",classification_report(y_actual,y))
print('Confusion Matrix \n' , confusion_matrix(y_actual,y))

In [ ]:
model.save('multiclass_classifier')